In [1]:
from pandas import DataFrame, read_csv, concat
import random
import math
from useGeneratedData import splitTrainTest, NewPredicter, makeDataUsable

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Testing data
size = 20
paramsToStore = {"Time": [0,1,2,3,0,1,2,3,4,5,0,1,2,3,4,5,6,7,8,9],
                "TTC": [random.uniform(0, 10) for _ in range(size)],
                "DTO": [random.uniform(0, 20) for _ in range(size)],
                "JERK": [abs(random.gauss(0, 20)) for _ in range(size)],
                "Speed": [random.uniform(0, 15) for _ in range(size)],
                "asX": [random.gauss(0, 0.15) for _ in range(size)],
                "asY": [random.gauss(0, 0.15) for _ in range(size)],
                "asZ": [random.gauss(0, 0.1) for _ in range(size)],
                "COL": [0,0,0,1,0,0,0,1,0,0, 0,0,0,1,0,0,0,1,0,0]}
df = DataFrame(paramsToStore)
# df.set_index("Time", inplace=True)
# df.to_csv(PATH+"/data/testGenerateData.csv", index=False)
df.head(20)

,Time,TTC,DTO,JERK,Speed,asX,asY,asZ,COL
0,0,7.925208,15.933437,25.645810,12.590772,0.025443,-0.180770,0.075587,0
1,1,7.002332,16.233319,40.208376,12.789863,0.032708,-0.025316,0.105092,0
2,2,3.767259,10.013044,42.024013,0.027384,-0.511466,0.105235,-0.043563,0
3,3,6.836008,9.931474,11.594839,5.322831,0.087182,0.147977,0.101152,1
4,0,0.219982,4.359584,53.014012,6.995083,-0.005002,0.027986,0.140186,0
5,1,2.555299,1.940769,27.639370,11.763641,-0.100998,0.316521,-0.088831,0
6,2,3.358577,17.158828,16.067957,0.169314,0.086034,-0.084619,-0.012201,0
7,3,5.633592,2.865899,26.744139,14.388324,0.009543,-0.077476,-0.115699,1
8,4,9.913071,5.499353,24.469136,10.913183,-0.216512,0.356399,-0.011747,0
9,5,2.333337,6.422581,6.450475,4.520565,0.293791,-0.234486,-0.042731,0


In [3]:
def makeDataUsable(df: DataFrame, pastImportance: int=6, rowsBeforeCol: int=3, removeCol: bool=True) -> DataFrame:
    """
    Make a usable Dataframe from a csv file from generated data.\\
    Creates pastImportance of columns for each prediction feature.\\
    Registers the "col" feature before an actual collision as 1, up to secBeforeCol back in time.\\
    If undersampleRatio is > 0, it will under sample the data, if it is 0, it will not.
    
    ### Params:
        * df: Dataframe, need to have the columns named "TTC", "DTO", "JERK" and "Speed"ArithmeticError
        * pastImportance: int, needs to be > 0
        * rowsBeforeCol: int, needs to be > 0
        * removeCol: bool, True if the origianl collision row should be removed
    ### Returns:
        * Dataframe
    """
    colsToUse = ["TTC", "DTO", "JERK", "Speed", "asX", "asY", "asZ"]
    columns = ["Time"]
    columns += [f"{c}{i}" for c in colsToUse for i in range(1, pastImportance+1)]
    columns += ["COL", "toRemove"]

    rowsToRemove = []
    
    dataDict = {col: [] for col in columns}

    for i, row in df.iterrows():
        if row["Time"] < pastImportance-1:
            continue

        dataDict["Time"].append(row["Time"])
        for j, k in enumerate(range(i-pastImportance+1, i+1), start=1):
            for c in colsToUse:
                # Should be able to remove this if check with new data
                if (c == "TTC" or c == "DTO") and df.iloc[k][c] < 0:
                    dataDict[f"{c}{j}"].append(0)
                    continue
                dataDict[f"{c}{j}"].append(df.iloc[k][c])

        dataDict["COL"].append(row["COL"])
        dataDict["toRemove"].append(row["COL"])
        if row["COL"] == 1:
            rowsToRemove.append(i-1)
            available = int(row["Time"]-pastImportance+2)
            amount = int(rowsBeforeCol) if available >= rowsBeforeCol else available
            dataDict["COL"][-amount:] = [1]*amount

    df = DataFrame(dataDict)
    df = df[df["toRemove"] == 0] if removeCol else df
    df.drop("toRemove", axis=1, inplace=True)
    return df

# makeDataUsable(read_csv("../../data/testGenerateData.csv"), pastImportance=4, secBeforeCol=4)

In [4]:
makeDataUsable(df, 2, 3, True)

,Time,TTC1,TTC2,DTO1,DTO2,JERK1,JERK2,Speed1,Speed2,asX1,asX2,asY1,asY2,asZ1,asZ2,COL
0,1.0,7.925208,7.002332,15.933437,16.233319,25.645810,40.208376,12.590772,12.789863,0.025443,0.032708,-0.180770,-0.025316,0.075587,0.105092,1.0
1,2.0,7.002332,3.767259,16.233319,10.013044,40.208376,42.024013,12.789863,0.027384,0.032708,-0.511466,-0.025316,0.105235,0.105092,-0.043563,1.0
3,1.0,0.219982,2.555299,4.359584,1.940769,53.014012,27.639370,6.995083,11.763641,-0.005002,-0.100998,0.027986,0.316521,0.140186,-0.088831,1.0
4,2.0,2.555299,3.358577,1.940769,17.158828,27.639370,16.067957,11.763641,0.169314,-0.100998,0.086034,0.316521,-0.084619,-0.088831,-0.012201,1.0
6,4.0,5.633592,9.913071,2.865899,5.499353,26.744139,24.469136,14.388324,10.913183,0.009543,-0.216512,-0.077476,0.356399,-0.115699,-0.011747,0.0
7,5.0,9.913071,2.333337,5.499353,6.422581,24.469136,6.450475,10.913183,4.520565,-0.216512,0.293791,0.356399,-0.234486,-0.011747,-0.042731,0.0
8,1.0,4.089684,0.781457,10.681662,10.684111,1.260866,10.981635,0.890700,4.512204,-0.135675,0.416428,-0.112562,-0.004654,-0.088185,-0.095902,1.0
9,2.0,0.781457,3.899687,10.684111,7.941733,10.981635,41.394334,4.512204,9.584366,0.416428,-0.202070,-0.004654,-0.023975,-0.095902,0.116757,1.0
11,4.0,0.562588,7.689322,5.542141,1.613677,10.474201,11.753614,6.046215,7.945042,-0.041899,-0.216730,0.263737,-0.061423,-0.020339,-0.146130,0.0
12,5.0,7.689322,6.538496,1.613677,7.640565,11.753614,17.430180,7.945042,6.023951,-0.216730,0.005194,-0.061423,0.019083,-0.146130,-0.196812,1.0


In [18]:
rawData = read_csv("../../data/generatedData.csv")
rawData.iloc[18:25]

,Time,TTC,DTO,JERK,Speed,asX,asY,asZ,COL
18,19.0,14.615,19.1452,1.447,13.282,0.000,0.000,0.000,0
19,20.0,17.307,18.2413,0.009,12.981,0.000,0.000,0.000,0
20,21.0,5.168,15.4067,0.009,12.689,0.000,0.000,0.000,0
21,22.0,0.247,3.0764,0.007,12.404,0.000,0.000,-0.000,0
22,23.0,0.030,0.2646,11.970,0.149,-0.017,-0.001,0.006,1
23,24.0,50.000,0.8598,12.106,0.000,0.000,0.000,0.000,0
24,25.0,50.000,1.4957,0.149,0.000,-0.000,0.000,0.000,0


In [22]:
len(rawData)//2/60

34.81666666666667

In [5]:
data = makeDataUsable(read_csv("../../data/generatedData.csv"), pastImportance=4, rowsBeforeCol=5, removeCol=True)
data.head(5)


,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
0,3.0,50.0,50.0,50.0,50.000,16.6264,100.0,100.0,100.0000,0.004,...,0.000,0.000,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,0.0
1,4.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,0.661,...,-0.001,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,-0.001,0.0
2,5.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,4.117,...,-0.000,-0.001,-0.001,0.000,0.000,0.001,0.000,-0.001,0.000,0.0
3,6.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,0.865,...,0.000,-0.001,0.000,0.000,0.000,0.000,-0.001,0.000,0.000,0.0
4,7.0,50.0,50.0,50.0,5.322,100.0000,100.0,100.0,83.3296,3.704,...,0.004,0.000,0.000,0.000,-0.001,-0.001,0.000,0.000,0.005,0.0


In [10]:
cc = read_csv("../../data/generatedData.csv")
collisions = len(data[data["COL"] == 1])

print(f"Amount of unique collisions: {len(cc[cc['COL'] == 1])}")
print(f"Amount of collisions: {len(data[data['COL'] == 1])}")
print(f"Amount not colliding: {len(data[data['COL'] == 0])}")

Amount of unique collisions: 215
Amount of collisions: 492
Amount not colliding: 3458


In [13]:
data.corr()["COL"]

Time     -0.065969
TTC1     -0.101218
TTC2     -0.177124
TTC3     -0.251498
TTC4     -0.316259
DTO1     -0.062557
DTO2     -0.109763
DTO3     -0.157293
DTO4     -0.208628
JERK1    -0.051574
JERK2    -0.043444
JERK3    -0.045155
JERK4    -0.068430
Speed1    0.154619
Speed2    0.170875
Speed3    0.174013
Speed4    0.168772
asX1     -0.010917
asX2     -0.007154
asX3     -0.014407
asX4     -0.017245
asY1      0.017051
asY2      0.018624
asY3      0.000970
asY4     -0.001754
asZ1      0.014642
asZ2      0.021415
asZ3      0.024463
asZ4      0.038726
COL       1.000000
Name: COL, dtype: float64

In [97]:
def getResultsFromData(filename: str, pastImportance: int=4, rowsBeforeCol: int=4, removeCol: bool=True, undersampleRatio: float=1):
    """
    TODO
    ### Params:
        * undersampleRatio: float, ratio of non collisions / collisions
    """
    data = makeDataUsable(read_csv(filename), pastImportance, rowsBeforeCol, removeCol)
    # dataToUse = data.copy()
    data.drop(columns=data.columns[0], axis=1, inplace=True) # Dropping 'Time'
    # print(data.head(10))
    # data.corr()["COL"]

    if undersampleRatio > 0:
        sampleSize = int(len(data[data['COL'] == 1]) * undersampleRatio)
        if sampleSize < len(data[data['COL'] == 0]):
            print("Undersampling")
            data = concat([data[data["COL"] == 0].sample(sampleSize, random_state=1), data[data["COL"] == True]])
    
    trainX, trainY, testX, testY = splitTrainTest(data, splitRatio=0.8)
    print(f"trainX: {trainX.shape}, trainY: {trainY.shape}, testX: {testX.shape}, testY: {testY.shape}")

    p = NewPredicter()
    trainXpp = p.preProcess(trainX)
    trainYpp = trainY.to_numpy()
    testXpp = p.preProcess(testX)
    testYpp = testY.to_numpy()

    p.fit(trainXpp, trainYpp)
    pred = p.predict(testXpp)
    cm = p.getScore(pred, testYpp)

    return p, data, cm

In [98]:
p, dataRemoveCol, cmRemoveCol = getResultsFromData("../../data/generatedData.csv", pastImportance=4, rowsBeforeCol=4, removeCol=True, undersampleRatio=1)
# dataRemoveCol.corr()["COL"]

Undersampling
Splitting at 598, total rows: 748
trainX: (598, 28), trainY: (598,), testX: (150, 28), testY: (150,)
Scaler is fitted
Total: 150, Collisions: 73
Accuracy: 0.6533333333333333, Precision: 0.6329113924050633, Recall: 0.684931506849315, F1: 0.6578947368421053
Confusion matrix:
[[48 29]
 [23 50]]


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [51]:
print("Feature   Corr to 'COL'   Min value    Max value")
for c, x, y, z in zip(dataRemoveCol.columns, dataRemoveCol.corr()["COL"], dataRemoveCol.min(), dataRemoveCol.max()):
    print(f"{c.ljust(12)} {str(round(x, 3)).ljust(12)} {str(round(y, 3)).ljust(12)} {str(round(z, 3)).ljust(12)}")


Feature   Corr to 'COL'   Min value    Max value
TTC1         -0.218       0.0          50.0        
TTC2         -0.301       0.0          50.0        
TTC3         -0.378       0.0          50.0        
TTC4         -0.467       0.0          50.0        
DTO1         -0.108       0.0          100.0       
DTO2         -0.223       0.0          100.0       
DTO3         -0.308       0.0          100.0       
DTO4         -0.399       0.0          100.0       
JERK1        -0.07        0.0          65.892      
JERK2        -0.04        0.0          85.116      
JERK3        -0.058       0.0          86.352      
JERK4        -0.095       0.0          33.483      
Speed1       0.214        0.0          32.73       
Speed2       0.212        0.0          34.721      
Speed3       0.202        0.0          32.73       
Speed4       0.185        0.0          34.721      
asX1         0.038        -0.463       0.233       
asX2         -0.049       -0.315       0.254       
asX3         -0

In [87]:
dataRemoveCol[['TTC1', 'TTC2', 'TTC3', 'TTC4', 'DTO1', 'DTO2', 'DTO3', 'DTO4', 'JERK1','JERK2', 'JERK3', 'JERK4', 'Speed1', 'Speed2', 'Speed3', 'Speed4', 'COL']].corr()
# dataRemoveCol[['TTC1', 'TTC2', 'TTC3', 'TTC4']]

,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,JERK3,JERK4,Speed1,Speed2,Speed3,Speed4,COL
TTC1,1.000000,0.572853,0.396297,0.286086,0.338424,0.323809,0.330951,0.292208,-0.017977,-0.146863,-0.084863,0.064879,-0.231838,-0.114824,-0.045497,-0.007800,-0.217747
TTC2,0.572853,1.000000,0.560639,0.428378,0.265085,0.387294,0.359175,0.347288,0.085890,-0.055062,-0.076778,0.124697,-0.320184,-0.209518,-0.146671,-0.099175,-0.300520
TTC3,0.396297,0.560639,1.000000,0.574980,0.097355,0.285341,0.390649,0.339234,0.091405,0.071057,0.034610,0.086277,-0.393839,-0.370772,-0.274672,-0.212991,-0.378448
TTC4,0.286086,0.428378,0.574980,1.000000,0.101534,0.194148,0.329302,0.439026,0.096947,0.069894,0.053752,0.141017,-0.341470,-0.349524,-0.346727,-0.285268,-0.467215
DTO1,0.338424,0.265085,0.097355,0.101534,1.000000,0.769329,0.603658,0.511128,-0.144968,-0.125680,-0.058936,-0.059500,0.398637,0.455911,0.457441,0.458051,-0.107781
DTO2,0.323809,0.387294,0.285341,0.194148,0.769329,1.000000,0.770722,0.625268,-0.116848,-0.114748,-0.065790,-0.036706,0.290372,0.364644,0.396546,0.413489,-0.222618
DTO3,0.330951,0.359175,0.390649,0.329302,0.603658,0.770722,1.000000,0.732186,-0.094765,-0.097740,-0.079688,-0.052941,0.151760,0.236185,0.305589,0.340691,-0.308187
DTO4,0.292208,0.347288,0.339234,0.439026,0.511128,0.625268,0.732186,1.000000,-0.049303,-0.043501,-0.017708,-0.017120,0.079263,0.134820,0.193323,0.242183,-0.399014
JERK1,-0.017977,0.085890,0.091405,0.096947,-0.144968,-0.116848,-0.094765,-0.049303,1.000000,0.389400,0.020966,0.082649,-0.234327,-0.228050,-0.204099,-0.186089,-0.069724
JERK2,-0.146863,-0.055062,0.071057,0.069894,-0.125680,-0.114748,-0.097740,-0.043501,0.389400,1.000000,0.662851,0.138306,-0.069479,-0.230282,-0.223485,-0.208238,-0.039798


In [35]:
p, dataNotRemoveCol, cmNotRemoveCol = getResultsFromData("../../data/generatedData.csv", pastImportance=4, rowsBeforeCol=4, removeCol=False, undersampleRatio=1)
# dataNotRemoveCol.corr()["COL"]

Undersampling
Splitting at 942, total rows: 1178
trainX: (942, 28), trainY: (942,), testX: (236, 28), testY: (236,)
Scaler is fitted
Total: 236, Collisions: 119
Accuracy: 0.7372881355932204, Precision: 0.728, Recall: 0.7647058823529411, F1: 0.7459016393442623
Confusion matrix:
[[83 34]
 [28 91]]


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [37]:
print("Feature, removed, not removed")
for c, x, y, in zip(dataRemoveCol.columns, dataRemoveCol.corr()["COL"], dataNotRemoveCol.corr()["COL"]):
    print(c, round(x, 2), round(y, 2))

Feature, removed, not removed
TTC1 -0.22 -0.3
TTC2 -0.3 -0.38
TTC3 -0.38 -0.45
TTC4 -0.47 -0.44
DTO1 -0.11 -0.2
DTO2 -0.22 -0.28
DTO3 -0.31 -0.36
DTO4 -0.4 -0.42
JERK1 -0.07 -0.07
JERK2 -0.04 -0.02
JERK3 -0.06 0.19
JERK4 -0.09 0.32
Speed1 0.21 0.24
Speed2 0.21 0.22
Speed3 0.2 0.11
Speed4 0.18 -0.06
asX1 0.04 0.04
asX2 -0.05 -0.01
asX3 -0.0 0.04
asX4 -0.05 0.02
asY1 0.01 0.04
asY2 0.07 -0.01
asY3 -0.0 -0.01
asY4 -0.04 -0.05
asZ1 0.03 0.06
asZ2 0.07 0.07
asZ3 0.01 -0.0
asZ4 0.05 -0.01
COL 1.0 1.0


In [42]:
# NOTE DO NOT RUN
# Expected time to run: ~53 minutes

pastImportances = [2,3,4,5,6,7]
rowsBeforeCols = [2,3,4,5,6,7]
removeCols = {False, True}
samplings = [1, 1.5, 2]

allCombos = [(pi, rbc, rc, sr) for pi in pastImportances for rbc in rowsBeforeCols for rc in removeCols for sr in samplings]

for (pi, rbc, rc, sr) in allCombos:
    print(f"\nPast importance: {pi}, sec before col: {rbc}, removeCol: {rc}, sample ratio: {sr}")
    _, _ = getResultsFromData("../../data/generatedData.csv", pastImportance=pi, rowsBeforeCol=rbc, removeCol=rc, undersampleRatio=sr)


Past importance: 2, sec before col: 2, removeCol: False, sample ratio: 1
Undersampling
Splitting at 552, total rows: 690
trainX: (552, 14), trainY: (552,), testX: (138, 14), testY: (138,)
Scaler is fitted


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Total: 138, Collisions: 73
Accuracy: 0.8623188405797102, Precision: 0.875, Recall: 0.863013698630137, F1: 0.8689655172413793
Confusion matrix:
[[56  9]
 [10 63]]

Past importance: 2, sec before col: 2, removeCol: False, sample ratio: 1.5
Undersampling
Splitting at 689, total rows: 862
trainX: (689, 14), trainY: (689,), testX: (173, 14), testY: (173,)
Scaler is fitted


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Total: 173, Collisions: 67
Accuracy: 0.8439306358381503, Precision: 0.7857142857142857, Recall: 0.8208955223880597, F1: 0.8029197080291971
Confusion matrix:
[[91 15]
 [12 55]]

Past importance: 2, sec before col: 2, removeCol: False, sample ratio: 2
Undersampling
Splitting at 828, total rows: 1035
trainX: (828, 14), trainY: (828,), testX: (207, 14), testY: (207,)
Scaler is fitted
Total: 207, Collisions: 71
Accuracy: 0.9033816425120773, Precision: 0.84, Recall: 0.8873239436619719, F1: 0.863013698630137
Confusion matrix:
[[124  12]
 [  8  63]]

Past importance: 2, sec before col: 2, removeCol: True, sample ratio: 1
Undersampling
Splitting at 208, total rows: 260
trainX: (208, 14), trainY: (208,), testX: (52, 14), testY: (52,)
Scaler is fitted
Total: 52, Collisions: 31
Accuracy: 0.7115384615384616, Precision: 0.7857142857142857, Recall: 0.7096774193548387, F1: 0.7457627118644068
Confusion matrix:
[[15  6]
 [ 9 22]]

Past importance: 2, sec before col: 2, removeCol: True, sample ratio: 1.5

...

Total: 290, Collisions: 133
Accuracy: 0.6517241379310345, Precision: 0.6095890410958904, Recall: 0.6691729323308271, F1: 0.6379928315412187
Confusion matrix:
[[100  57]
 [ 44  89]]

Past importance: 4, sec before col: 7, removeCol: True, sample ratio: 1.5
Undersampling
Splitting at 1445, total rows: 1807
trainX: (1445, 28), trainY: (1445,), testX: (362, 28), testY: (362,)
Scaler is fitted
Total: 362, Collisions: 123
Accuracy: 0.6988950276243094, Precision: 0.5448717948717948, Recall: 0.6910569105691057, F1: 0.6093189964157706
Confusion matrix:
[[168  71]
 [ 38  85]]

Past importance: 4, sec before col: 7, removeCol: True, sample ratio: 2
Undersampling
Splitting at 1735, total rows: 2169
trainX: (1735, 28), trainY: (1735,), testX: (434, 28), testY: (434,)
Scaler is fitted
Total: 434, Collisions: 136
Accuracy: 0.7142857142857143, Precision: 0.5416666666666666, Recall: 0.5735294117647058, F1: 0.557142857142857
Confusion matrix:
[[232  66]
 [ 58  78]]

Past importance: 5, sec before col: 2

c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Total: 244, Collisions: 125
Accuracy: 0.6352459016393442, Precision: 0.6451612903225806, Recall: 0.64, F1: 0.642570281124498
Confusion matrix:
[[75 44]
 [45 80]]

Past importance: 5, sec before col: 6, removeCol: True, sample ratio: 1.5
Undersampling
Splitting at 1217, total rows: 1522
trainX: (1217, 35), trainY: (1217,), testX: (305, 35), testY: (305,)
Scaler is fitted
Total: 305, Collisions: 122
Accuracy: 0.7311475409836066, Precision: 0.6538461538461539, Recall: 0.6967213114754098, F1: 0.6746031746031745
Confusion matrix:
[[138  45]
 [ 37  85]]

Past importance: 5, sec before col: 6, removeCol: True, sample ratio: 2
Undersampling
Splitting at 1461, total rows: 1827
trainX: (1461, 35), trainY: (1461,), testX: (366, 35), testY: (366,)
Scaler is fitted
Total: 366, Collisions: 108
Accuracy: 0.726775956284153, Precision: 0.5294117647058824, Recall: 0.6666666666666666, F1: 0.5901639344262295
Confusion matrix:
[[194  64]
 [ 36  72]]

Past importance: 5, sec before col: 7, removeCol: False,

c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Total: 290, Collisions: 141
Accuracy: 0.6724137931034483, Precision: 0.6575342465753424, Recall: 0.6808510638297872, F1: 0.6689895470383275
Confusion matrix:
[[99 50]
 [45 96]]

Past importance: 5, sec before col: 7, removeCol: True, sample ratio: 1.5
Undersampling
Splitting at 1445, total rows: 1807
trainX: (1445, 35), trainY: (1445,), testX: (362, 35), testY: (362,)
Scaler is fitted
Total: 362, Collisions: 144
Accuracy: 0.712707182320442, Precision: 0.6282051282051282, Recall: 0.6805555555555556, F1: 0.6533333333333333
Confusion matrix:
[[160  58]
 [ 46  98]]

Past importance: 5, sec before col: 7, removeCol: True, sample ratio: 2
Undersampling
Splitting at 1735, total rows: 2169
trainX: (1735, 35), trainY: (1735,), testX: (434, 35), testY: (434,)
Scaler is fitted
Total: 434, Collisions: 143
Accuracy: 0.7396313364055299, Precision: 0.6041666666666666, Recall: 0.6083916083916084, F1: 0.6062717770034843
Confusion matrix:
[[234  57]
 [ 56  87]]

Past importance: 6, sec before col: 2, re

In [16]:
# data = makeDataUsable(read_csv("../../data/testGenerateData.csv"), 6, 4)
dataToUse = data.copy()
dataToUse.drop(columns=data.columns[0], axis=1, inplace=True)
# print(data.head(10))
trainX, trainY, testX, testY = splitTrainTest(dataToUse)
print(f"trainX:{trainX.shape}, trainY:{trainY.shape}, testX:{testX.shape}, testY:{testY.shape}")
# print(trainX.head())
# print(f"trainX: {type(trainX)}, trainY: {type(trainY)}, testX: {type(testX)}, testY: {type(testY)}")


splitting at 215.
trainX:(215, 42), trainY:(215,), testX:(54, 42), testY:(54,)
trainX: <class 'pandas.core.frame.DataFrame'>, trainY: <class 'pandas.core.series.Series'>, testX: <class 'pandas.core.frame.DataFrame'>, testY: <class 'pandas.core.series.Series'>


In [17]:

# p = NewPredicter.loadModel("xgb_2_582-11-16-201")
# print(p.__dict__)

p = NewPredicter()
trainXpp = p.preProcess(trainX)
trainYpp = trainY.to_numpy()
testXpp = p.preProcess(testX)
testYpp = testY.to_numpy()
print(f"trainX: {type(trainXpp)}, trainY: {type(trainYpp)}, testX: {type(testXpp)}, testY: {type(testYpp)}")
# print(trainXpp)
p.fit(trainXpp, trainYpp)

# p.predict([])

Scaler is fitted
trainX: <class 'numpy.ndarray'>, trainY: <class 'numpy.ndarray'>, testX: <class 'numpy.ndarray'>, testY: <class 'numpy.ndarray'>


In [18]:
pred = []
for i in range(len(testX)):
    pred.append(p.predict(testX.iloc[i]))
print(pred)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [19]:
p.getScore(pred, testY)

Total: 54, number of collisions: 14
	TN: 36 	| FP: 4 
	FN: 4 	| TP: 10
Accuracy: 0.85
Precision: 0.71
Recall: 0.71
F1: 0.71


[[36, 4], [4, 10]]